根据`analyze.md`的内容，处理运算数据库数据。

处理目标数据库：invite_info两库。

计算train与test时计算每个人`关注话题命中数`、`感兴趣话题与此话题加权正余弦距离和`、`邀请天数间隔`、`邀请小时`。

In [14]:
import pandas as pd
import pymysql
from tqdm import tqdm

In [34]:
BATCH_SIZE = 50000  # 每次从数据库最多拿多少数据
DB_NAME = 'zhihu2019_dataset'

In [16]:
class DB():
    def __init__(self, host='localhost', port=3306, db=DB_NAME, user='root', passwd='123456', charset='utf8'):
        self.conn = pymysql.connect(host=host, port=port, db=db, user=user, passwd=passwd, charset=charset)   
        self.cur = self.conn.cursor(cursor = pymysql.cursors.DictCursor)

    def __enter__(self): 
        return self.cur

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.conn.commit()
        self.cur.close()
        self.conn.close()

## 1 关注话题命中数

查看已处理数据数。

In [32]:
# 查看已经处理了多少数据
with DB(db=DB_NAME) as db:
    db.execute('select count(*) as length from invite_info_0926')
    length_all = db.fetchone()['length']
    db.execute('select count(*) as length from invite_info_0926 where 关注话题命中数=-1')
    length_need_do = db.fetchone()['length']
    print('%d / %d' % (length_all - length_need_do, length_all))

20016 / 9489162


处理函数。

In [17]:
# 计算某行关注话题命中数
def get_aim(qid, uid, qtid, utid):
    qtid = [] if qtid == -1 else qtid.split(',')
    utid = [] if utid == -1 else utid.split(',')
    qtid.extend(utid)
    aim = len(qtid) - len(set(qtid))
    return aim

In [27]:
# 批量更新表
def batch_update(db, table, data):
    length = len(data)
    data['values'] = data[['问题ID', '用户ID', '关注话题命中数']].apply(
        lambda x: '(\'%s\',\'%s\',%d)' % (x[0], x[1], x[2]), axis=1)
    values = ','.join(data['values'].values)
    
    sql_1 = '''CREATE TEMPORARY TABLE tmp (
            问题ID VARCHAR (20),
            用户ID VARCHAR (20),
            关注话题命中数 INT (11)
        );'''
    sql_2 = 'ALTER TABLE tmp ADD INDEX tmp_index(问题ID, 用户ID);'
    sql_3 = '''INSERT INTO tmp
                VALUES %s;''' % values
    sql_4 = '''UPDATE %s t, tmp
                SET t.关注话题命中数 = tmp.关注话题命中数
                WHERE
                    t.问题ID = tmp.问题ID
                AND t.用户ID = tmp.用户ID;''' % table
    sql_5 = 'DROP TABLE tmp;'
    
    db.execute(sql_1)
    db.execute(sql_2)
    db.execute(sql_3)
    db.execute(sql_4)
    db.execute(sql_5)

**处理数据并更新。**

In [34]:
tables = ['invite_info_0926', 'invite_info_evaluate_1_0926']
for table in tables:
    with DB(db=DB_NAME) as db:
        sql = 'select count(*) as length from %s where 关注话题命中数=-1' % table
        db.execute(sql)
        result = db.fetchone()
        with tqdm(total=result['length']) as pbar:
            while True:
                # 读取一些数据
                sql = '''SELECT
                        t.问题ID,
                        t.用户ID,
                        q.问题绑定话题ID,
                        m.关注话题
                    FROM
                        (
                            (
                                SELECT
                                    问题ID,
                                    用户ID
                                FROM
                                    %s
                                WHERE
                                    关注话题命中数 =- 1
                                LIMIT %d
                            ) t
                            INNER JOIN (
                                SELECT
                                    问题ID,
                                    问题绑定话题ID
                                FROM
                                    question_info_0926
                            ) q ON t.问题ID = q.问题ID
                        )
                    INNER JOIN (
                        SELECT
                            用户ID,
                            关注话题
                        FROM
                            member_info_0926
                    ) m ON t.用户ID = m.用户ID''' % (table, BATCH_SIZE)
                db.execute(sql)
                result = db.fetchall()
                data = pd.DataFrame(result)
                if len(data) == 0:
                    break
                
                # 计算关注话题命中数
                data['关注话题命中数'] = data[['问题ID', '用户ID', '问题绑定话题ID', '关注话题']].apply(
                    lambda x: get_aim(x[0], 
                                       x[1], 
                                       x[2], 
                                       x[3]), 
                    axis=1)
                
                # 写入数据库
                batch_update(db, table, data)
                
                # 刷新进度条
                pbar.update(len(data))


100%|█████████▉| 1141630/1141683 [05:10<00:00, 3489.87it/s]


## 2 感兴趣话题命中

In [41]:
# 计算某行感兴趣话题命中数
def get_aim(qtid, utid, pbar):
    if qtid == '-1' or utid == '-1':
        pbar.update(1)
        return 0
    
    qtid = qtid.split(',')
    utid = utid.split(',')
    utid = [float(x.split(':')[1]) for x in utid if x.split(':')[0] in qtid]
    aim = sum(utid)
    
    pbar.update(1)
    if aim > 100:
        return 0
    return aim

In [36]:
# 批量更新表
def batch_update(db, table, data):
    data['values'] = data[['问题ID', '用户ID', '感兴趣话题命中率']].apply(
        lambda x: '(\'%s\',\'%s\',%f)' % (x[0], x[1], x[2]), axis=1)
    values = ','.join(data['values'].values)
    
    sql_1 = '''CREATE TEMPORARY TABLE tmp (
            问题ID VARCHAR (20),
            用户ID VARCHAR (20),
            感兴趣话题命中率 float
        );'''
    sql_2 = 'ALTER TABLE tmp ADD INDEX tmp_index(问题ID, 用户ID);'
    sql_3 = '''INSERT INTO tmp
                VALUES %s;''' % values
    sql_4 = '''UPDATE %s t, tmp
                SET t.感兴趣话题命中率 = tmp.感兴趣话题命中率
                WHERE
                    t.问题ID = tmp.问题ID
                AND t.用户ID = tmp.用户ID;''' % table
    sql_5 = 'DROP TABLE tmp;'
    
    db.execute(sql_1)
    db.execute(sql_2)
    db.execute(sql_3)
    db.execute(sql_4)
    db.execute(sql_5)

In [44]:
tables = ['invite_info_0926', 'invite_info_evaluate_1_0926']
for table in tables:
    with DB(db=DB_NAME) as db:
        sql = 'select count(*) as length from %s where 感兴趣话题命中率=-1' % table
        db.execute(sql)
        result = db.fetchone()
        with tqdm(total=result['length']) as pbar:
            while True:
                # 读取一些数据
                sql = '''SELECT
                        t.问题ID,
                        t.用户ID,
                        q.问题绑定话题ID,
                        m.感兴趣话题
                    FROM
                        (
                            (
                                SELECT
                                    问题ID,
                                    用户ID
                                FROM
                                    %s
                                WHERE
                                    感兴趣话题命中率 =- 1
                                LIMIT %d
                            ) t
                            INNER JOIN (
                                SELECT
                                    问题ID,
                                    问题绑定话题ID
                                FROM
                                    question_info_0926
                            ) q ON t.问题ID = q.问题ID
                        )
                    INNER JOIN (
                        SELECT
                            用户ID,
                            感兴趣话题
                        FROM
                            member_info_0926
                    ) m ON t.用户ID = m.用户ID''' % (table, BATCH_SIZE)
                db.execute(sql)
                result = db.fetchall()
                data = pd.DataFrame(result)
                if len(data) == 0:
                    break
                
                # 计算关注话题命中数
                data['感兴趣话题命中率'] = data[['问题ID', '用户ID', '问题绑定话题ID', '感兴趣话题']].apply(
                    lambda x: get_aim(x[2], 
                                       x[3],
                                       pbar), 
                    axis=1)
                
                # 写入数据库
                batch_update(db, table, data)


100%|█████████▉| 1141599/1141683 [04:59<00:00, 3812.98it/s] 


## 3 邀请天数间隔（感觉是无用数据）

```mysql
CREATE TEMPORARY TABLE tmp AS (
	SELECT
		i.问题ID,
		i.用户ID,
		i.邀请创建时间,
		SUBSTRING_INDEX(
			SUBSTRING_INDEX(
				i.邀请创建时间,
				'H' ,- 2
			),
			'-',
			1
		) - SUBSTRING_INDEX(
			SUBSTRING_INDEX(
				q.问题创建时间,
				'H' ,- 2
			),
			'-',
			1
		) AS 邀请天数间隔
	FROM
		invite_info_0926 i
	INNER JOIN question_info_0926 q ON i.问题ID = q.问题ID
);

UPDATE invite_info_0926 i,
 tmp
SET i.`邀请天数间隔` = tmp.邀请天数间隔
WHERE
	i.用户ID = tmp.用户ID
AND i.问题ID = tmp.问题ID
AND i.邀请创建时间 = tmp.邀请创建时间;

DROP TABLE tmp;
```

## 4 邀请小时

```mysql
update invite_info_0926
set 邀请小时=SUBSTRING_INDEX(邀请创建时间,'H',-1)
```

## 5 计算均值标准差

```mysql
INSERT INTO statistic_data
VALUES
	(
		'invite_info',
		'邀请小时',
		(
			SELECT
				avg(邀请小时)
			FROM
				invite_info_0926
		),
		(
			SELECT
				std(邀请小时)
			FROM
				invite_info_0926
		)
	)
    ```

## 6 整合数据

```mysql
CREATE TEMPORARY TABLE tmp AS (
	SELECT
		i.`问题ID` q_id,
		i.`用户ID` u_id,
		i.`是否回答` do_answer,
		i.`邀请小时` invite_hour,
		i.`关注话题命中数` follow_topic_hit,
		i.`感兴趣话题命中率` interest_topic_hit,
		u.`性别` sex,
		u.`访问频率` access_freq,
		u.`用户二分类特征A` bin_feat_a,
		u.`用户二分类特征B` bin_feat_b,
		u.`用户二分类特征C` bin_feat_c,
		u.`用户二分类特征D` bin_feat_d,
		u.`用户二分类特征E` bin_feat_e,
		u.`用户分类特征A` multi_feat_a,
		u.`用户分类特征B` multi_feat_b,
		u.`用户分类特征C` multi_feat_c,
		u.`用户分类特征D` multi_feat_d,
		u.`用户分类特征E` multi_feat_e,
		u.`盐值分数` salt,
		u.`回答数` u_answer,
		u.`优秀回答数` u_good_answer,
		u.`推荐回答数` u_recommand_answer,
		u.`包含图片回答比例` u_image_answer,
		u.`包含视频回答比例` u_video_answer,
		u.`回答内容字数平均数` u_word_avg,
		u.`回答收到点赞平均数` u_like_avg,
		u.`回答收到取赞平均数` u_unlike_avg,
		u.`回答收到评论平均数` u_comment_avg,
		u.`回答被收藏平均数` u_collect_avg,
		u.`回答收到感谢平均数` u_thanks_avg,
		u.`回答被举报平均数` u_report_avg,
		u.`回答收到没有帮助平均数` u_nohelp_avg,
		u.`回答收到反对平均数` u_oppose_avg,
		q.`回答数` q_answer,
		q.`优秀回答数` q_good_answer,
		q.`推荐回答数` q_recommand_answer,
		q.`包含图片回答数` q_image_answer,
		q.`包含视频回答数` q_video_answer,
		q.`回答内容字数平均数` q_word_avg,
		q.`回答总点赞数` q_like,
		q.`回答总评论数` q_comment,
		q.`回答总收藏数` q_collect,
		q.`回答总感谢数` q_thanks
	FROM
		(
			invite_info_0926 i
			INNER JOIN member_info_0926 u ON i.`用户ID` = u.`用户ID`
		)
	INNER JOIN question_info_0926 q ON i.`问题ID` = q.`问题ID`
);

UPDATE train a,
 tmp b
SET a.access_freq = b.access_freq,
 a.bin_feat_a = b.bin_feat_a,
 a.bin_feat_b = b.bin_feat_b,
 a.bin_feat_c = b.bin_feat_c,
 a.bin_feat_d = b.bin_feat_d,
 a.bin_feat_e = b.bin_feat_e,
 a.do_answer = b.do_answer,
 a.follow_topic_hit = b.follow_topic_hit,
 a.interest_topic_hit = b.interest_topic_hit,
 a.invite_hour = b.invite_hour,
 a.multi_feat_a = b.multi_feat_a,
 a.multi_feat_b = b.multi_feat_b,
 a.multi_feat_c = b.multi_feat_c,
 a.multi_feat_d = b.multi_feat_d,
 a.multi_feat_e = b.multi_feat_e,
 a.q_answer = b.q_answer,
 a.q_collect = b.q_collect,
 a.q_comment = b.q_comment,
 a.q_good_answer = b.q_good_answer,
 a.q_id = b.q_id,
 a.q_image_answer = b.q_image_answer,
 a.q_like = b.q_like,
 a.q_recommand_answer = b.q_recommand_answer,
 a.q_thanks = b.q_thanks,
 a.q_video_answer = b.q_video_answer,
 a.q_word_avg = b.q_word_avg
 ```

二代：

```mysql
CREATE TABLE train AS (
	SELECT
		i.`问题ID` q_id,
		i.`用户ID` u_id,
		i.`是否回答` do_answer,
		i.`邀请小时` invite_hour,
		i.`关注话题命中数` follow_topic_hit,
		i.`感兴趣话题命中率` interest_topic_hit,
		u.`性别` sex,
		u.`访问频率` access_freq,
		u.`用户二分类特征A` bin_feat_a,
		u.`用户二分类特征B` bin_feat_b,
		u.`用户二分类特征C` bin_feat_c,
		u.`用户二分类特征D` bin_feat_d,
		u.`用户二分类特征E` bin_feat_e,
		u.`用户分类特征A` multi_feat_a,
		u.`用户分类特征B` multi_feat_b,
		u.`用户分类特征C` multi_feat_c,
		u.`用户分类特征D` multi_feat_d,
		u.`用户分类特征E` multi_feat_e,
		u.`盐值分数` salt,
		u.`回答数` u_answer,
		u.`优秀回答数` u_good_answer,
		u.`推荐回答数` u_recommand_answer,
		u.`包含图片回答比例` u_image_answer,
		u.`包含视频回答比例` u_video_answer,
		u.`回答内容字数平均数` u_word_avg,
		u.`回答收到点赞平均数` u_like_avg,
		u.`回答收到取赞平均数` u_unlike_avg,
		u.`回答收到评论平均数` u_comment_avg,
		u.`回答被收藏平均数` u_collect_avg,
		u.`回答收到感谢平均数` u_thanks_avg,
		u.`回答被举报平均数` u_report_avg,
		u.`回答收到没有帮助平均数` u_nohelp_avg,
		u.`回答收到反对平均数` u_oppose_avg,
		q.`回答数` q_answer,
		q.`优秀回答数` q_good_answer,
		q.`推荐回答数` q_recommand_answer,
		q.`包含图片回答数` q_image_answer,
		q.`包含视频回答数` q_video_answer,
		q.`回答内容字数平均数` q_word_avg,
		q.`回答总点赞数` q_like,
		q.`回答总评论数` q_comment,
		q.`回答总收藏数` q_collect,
		q.`回答总感谢数` q_thanks
	FROM
		(
			invite_info_0926 i
			LEFT JOIN member_info_0926 u USING (`用户ID`)
		)
	LEFT JOIN question_info_0926 q USING (问题ID)
);

ALTER TABLE train MODIFY do_answer INT;
ALTER TABLE train MODIFY invite_hour FLOAT;
ALTER TABLE train MODIFY follow_topic_hit FLOAT;
ALTER TABLE train MODIFY interest_topic_hit FLOAT;
ALTER TABLE train MODIFY access_freq VARCHAR (10);
ALTER TABLE train MODIFY bin_feat_a FLOAT;
ALTER TABLE train MODIFY bin_feat_b FLOAT;
ALTER TABLE train MODIFY bin_feat_c FLOAT;
ALTER TABLE train MODIFY bin_feat_d FLOAT;
ALTER TABLE train MODIFY bin_feat_e FLOAT;
ALTER TABLE train MODIFY multi_feat_a VARCHAR (20);
ALTER TABLE train MODIFY multi_feat_b VARCHAR (20);
ALTER TABLE train MODIFY multi_feat_c VARCHAR (20);
ALTER TABLE train MODIFY multi_feat_d VARCHAR (20);
ALTER TABLE train MODIFY multi_feat_e VARCHAR (20);
ALTER TABLE train MODIFY salt FLOAT;
ALTER TABLE train MODIFY u_answer FLOAT;
ALTER TABLE train MODIFY u_good_answer FLOAT;
ALTER TABLE train MODIFY u_recommand_answer FLOAT;
ALTER TABLE train MODIFY u_image_answer FLOAT;
ALTER TABLE train MODIFY u_video_answer FLOAT;
ALTER TABLE train MODIFY u_word_avg FLOAT;
ALTER TABLE train MODIFY u_like_avg FLOAT;
ALTER TABLE train MODIFY u_unlike_avg FLOAT;
ALTER TABLE train MODIFY u_comment_avg FLOAT;
ALTER TABLE train MODIFY u_collect_avg FLOAT;
ALTER TABLE train MODIFY u_thanks_avg FLOAT;
ALTER TABLE train MODIFY u_report_avg FLOAT;
ALTER TABLE train MODIFY u_nohelp_avg FLOAT;
ALTER TABLE train MODIFY u_oppose_avg FLOAT;
ALTER TABLE train MODIFY q_answer FLOAT;
ALTER TABLE train MODIFY q_good_answer FLOAT;
ALTER TABLE train MODIFY q_recommand_answer FLOAT;
ALTER TABLE train MODIFY q_image_answer FLOAT;
ALTER TABLE train MODIFY q_video_answer FLOAT;
ALTER TABLE train MODIFY q_word_avg FLOAT;
ALTER TABLE train MODIFY q_like FLOAT;
ALTER TABLE train MODIFY q_comment FLOAT;
ALTER TABLE train MODIFY q_collect FLOAT;
ALTER TABLE train MODIFY q_thanks FLOAT;
```

## 附 语句生成器

In [1]:
cols = ['invite_hour', 'follow_topic_hit', 'interest_topic_hit', 'bin_feat_a', 'bin_feat_b',
        'bin_feat_c', 'bin_feat_d', 'bin_feat_e', 'salt', 'u_answer', 'u_good_answer',
        'u_recommand_answer', 'u_image_answer', 'u_video_answer', 'u_word_avg', 'u_like_avg',
        'u_unlike_avg', 'u_comment_avg', 'u_collect_avg', 'u_thanks_avg', 'u_report_avg',
        'u_nohelp_avg', 'u_oppose_avg', 'q_answer', 'q_good_answer', 'q_recommand_answer',
        'q_image_answer', 'q_video_answer', 'q_word_avg', 'q_like', 'q_comment', 'q_collect',
        'q_thanks']

`train`数据集标准化

In [6]:
for i in cols:
    print('UPDATE train SET train.%s = (train.%s - (select mean from statistic_data where prop=\'%s\')) / (select std from statistic_data where prop=\'%s\');'
          % (i,i,i,i))

UPDATE train SET train.invite_hour = (train.invite_hour - (select mean from statistic_data where prop='invite_hour')) / (select std from statistic_data where prop='invite_hour');
UPDATE train SET train.follow_topic_hit = (train.follow_topic_hit - (select mean from statistic_data where prop='follow_topic_hit')) / (select std from statistic_data where prop='follow_topic_hit');
UPDATE train SET train.interest_topic_hit = (train.interest_topic_hit - (select mean from statistic_data where prop='interest_topic_hit')) / (select std from statistic_data where prop='interest_topic_hit');
UPDATE train SET train.bin_feat_a = (train.bin_feat_a - (select mean from statistic_data where prop='bin_feat_a')) / (select std from statistic_data where prop='bin_feat_a');
UPDATE train SET train.bin_feat_b = (train.bin_feat_b - (select mean from statistic_data where prop='bin_feat_b')) / (select std from statistic_data where prop='bin_feat_b');
UPDATE train SET train.bin_feat_c = (train.bin_feat_c - (select m

`val`数据集标准化

In [3]:
for i in cols:
    print('UPDATE val SET val.%s = (val.%s - (select mean from statistic_data where prop=\'%s\')) / (select std from statistic_data where prop=\'%s\');'
          % (i,i,i,i))

UPDATE val SET val.invite_hour = (val.invite_hour - (select mean from statistic_data where prop='invite_hour')) / (select std from statistic_data where prop='invite_hour');
UPDATE val SET val.follow_topic_hit = (val.follow_topic_hit - (select mean from statistic_data where prop='follow_topic_hit')) / (select std from statistic_data where prop='follow_topic_hit');
UPDATE val SET val.interest_topic_hit = (val.interest_topic_hit - (select mean from statistic_data where prop='interest_topic_hit')) / (select std from statistic_data where prop='interest_topic_hit');
UPDATE val SET val.bin_feat_a = (val.bin_feat_a - (select mean from statistic_data where prop='bin_feat_a')) / (select std from statistic_data where prop='bin_feat_a');
UPDATE val SET val.bin_feat_b = (val.bin_feat_b - (select mean from statistic_data where prop='bin_feat_b')) / (select std from statistic_data where prop='bin_feat_b');
UPDATE val SET val.bin_feat_c = (val.bin_feat_c - (select mean from statistic_data where prop='